In [ ]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Loading the dataset
train_dataset = pd.read_csv('train.csv')
train_dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
# Selecting features
features = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2',
            'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'HalfBath',
            'BedroomAbvGr', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
            'PoolArea', 'YrSold']
X = train_dataset[features]
y = train_dataset["SalePrice"]

In [ ]:
# Splitting the dataset into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# LightGBM objective function for Optuna
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100)
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    return rmse

In [ ]:
# Running the hyperparameter optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2023-07-12 14:28:09,289] A new study created in memory with name: no-name-5f21b65c-348f-4a4e-a08c-024be07b6a81
<ipython-input-7-bd34402695c0>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-7-bd34402695c0>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
<ipython-input-7-bd34402695c0>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_log

[LightGBM] [Warning] bagging_fraction is set=0.18831047342229987, subsample=1.0 will be ignored. Current value: bagging_fraction=0.18831047342229987
[LightGBM] [Warning] lambda_l2 is set=2.2658829529891475e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.2658829529891475e-06
[LightGBM] [Warning] lambda_l1 is set=0.0001162018346307885, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0001162018346307885
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8349045546477206, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8349045546477206


[I 2023-07-12 14:28:10,017] Trial 0 finished with value: 64036.116070974604 and parameters: {'lambda_l1': 0.0001162018346307885, 'lambda_l2': 2.2658829529891475e-06, 'num_leaves': 195, 'learning_rate': 0.014222768877965124, 'feature_fraction': 0.8349045546477206, 'bagging_fraction': 0.18831047342229987, 'bagging_freq': 2, 'min_child_samples': 91}. Best is trial 0 with value: 64036.116070974604.
<ipython-input-7-bd34402695c0>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-7-bd34402695c0>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8,

[LightGBM] [Warning] bagging_fraction is set=0.6444694686982728, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6444694686982728
[LightGBM] [Warning] lambda_l2 is set=1.8265469401425536, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8265469401425536
[LightGBM] [Warning] lambda_l1 is set=3.0805645312470566e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.0805645312470566e-06
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8249884854576126, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8249884854576126


[I 2023-07-12 14:28:10,265] Trial 1 finished with value: 41740.037847868145 and parameters: {'lambda_l1': 3.0805645312470566e-06, 'lambda_l2': 1.8265469401425536, 'num_leaves': 76, 'learning_rate': 0.021503463602538155, 'feature_fraction': 0.8249884854576126, 'bagging_fraction': 0.6444694686982728, 'bagging_freq': 2, 'min_child_samples': 51}. Best is trial 1 with value: 41740.037847868145.
<ipython-input-7-bd34402695c0>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-7-bd34402695c0>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0

...

[I 2023-07-12 14:28:46,377] Trial 97 finished with value: 28847.21204146541 and parameters: {'lambda_l1': 0.039850425111013076, 'lambda_l2': 3.3301610938046604e-07, 'num_leaves': 182, 'learning_rate': 0.06581657418559764, 'feature_fraction': 0.45649390869603507, 'bagging_fraction': 0.8722685024825839, 'bagging_freq': 10, 'min_child_samples': 8}. Best is trial 93 with value: 25833.169059244683.
<ipython-input-7-bd34402695c0>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-7-bd34402695c0>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 

[LightGBM] [Warning] bagging_fraction is set=0.9455421856385503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9455421856385503
[LightGBM] [Warning] lambda_l2 is set=9.269941922754347e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.269941922754347e-07
[LightGBM] [Warning] lambda_l1 is set=0.09977367479851128, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.09977367479851128
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.40253192006137634, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40253192006137634


[I 2023-07-12 14:28:46,937] Trial 98 finished with value: 27213.329137304652 and parameters: {'lambda_l1': 0.09977367479851128, 'lambda_l2': 9.269941922754347e-07, 'num_leaves': 173, 'learning_rate': 0.07434390641441621, 'feature_fraction': 0.40253192006137634, 'bagging_fraction': 0.9455421856385503, 'bagging_freq': 10, 'min_child_samples': 5}. Best is trial 93 with value: 25833.169059244683.
<ipython-input-7-bd34402695c0>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
<ipython-input-7-bd34402695c0>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1

[LightGBM] [Warning] bagging_fraction is set=0.8502075680120728, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8502075680120728
[LightGBM] [Warning] lambda_l2 is set=1.796483574235435e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.796483574235435e-07
[LightGBM] [Warning] lambda_l1 is set=0.05527198267184064, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.05527198267184064
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.48532052268317916, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48532052268317916


[I 2023-07-12 14:28:47,279] Trial 99 finished with value: 27627.22032223094 and parameters: {'lambda_l1': 0.05527198267184064, 'lambda_l2': 1.796483574235435e-07, 'num_leaves': 163, 'learning_rate': 0.0813799768386652, 'feature_fraction': 0.48532052268317916, 'bagging_fraction': 0.8502075680120728, 'bagging_freq': 10, 'min_child_samples': 11}. Best is trial 93 with value: 25833.169059244683.


In [ ]:
# Getting the best hyperparameters
best_params = study.best_params

In [ ]:
# Training the final LightGBM model with the best hyperparameters
final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(X, y)

[LightGBM] [Warning] lambda_l1 is set=0.10799834333787468, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.10799834333787468
[LightGBM] [Warning] feature_fraction is set=0.49015734367353314, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.49015734367353314
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] lambda_l2 is set=1.0706942293053998e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0706942293053998e-07
[LightGBM] [Warning] bagging_fraction is set=0.9450445406298583, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9450445406298583


LGBMRegressor(bagging_fraction=0.9450445406298583, bagging_freq=9,
              feature_fraction=0.49015734367353314,
              lambda_l1=0.10799834333787468, lambda_l2=1.0706942293053998e-07,
              learning_rate=0.07438082072642226, min_child_samples=3,
              num_leaves=143)